# **Proyecto: Sistema integrado de gestión y recomendación de viajes**
### **Materia:** Base de Datos 2025
## Alumno: Delfina González

Este trabajo simula la integración de datos diferentes bases de datos para modelar un sistema de gestión y recomendación de viajes. El sistema **almacena** datos de usuarios, destinos, hoteles, actividades, y realiza **recomendaciones** personalizadas a partir de datos distribuidos en **Neo4j, MongoDB y Redis.**

*En el mundo actual, los viajes y el turismo generan grandes cantidades de información sobre usuarios, destinos, alojamientos y actividades. Las empresas turísticas necesitan sistemas que no solo registren esta información, sino que también permitan realizar análisis, recomendaciones personalizadas y gestionar reservas de manera eficiente.*

Funcionalidades:
 - Almacenar información de usuarios,  historial de reservas, destinos turísticos, hoteles, actividades disponibles y precios asociados. 
 - Gestionar datos temporales como búsquedas recientes o reservas en proceso. 
 - Relaciones de conocimiento entre usuarios, relaciones de usuarios y destinos. 

Estructura del proyecto:

    - PARTE 0: "Configuración y conexiones"
    - PARTE A: "Carga inicial de Datasets"
    - PARTE B: "Consultas"
    - PARTE C: "Estadísticas y Gráficos"
    - PARTE D: "Modificaciones"
    - PARTE F: "Cierre de sesiones"

# PARTE 0: Configuración y conexiones

    Se cargan las bibliotecas
    Se establecen las conexiones con Neo4j, MongoDB y Redis.

In [671]:
import os, time
import json
import random
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from dotenv import load_dotenv
from neo4j import GraphDatabase
from pymongo import MongoClient
import redis
from datetime import datetime

# --- Configuración ---
load_dotenv()
print("Esperando servicios (5s)...")
time.sleep(5)

NEO4J_HOST = "bolt://neo4j:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4j123")
MONGO_URI = f"mongodb://{os.getenv('MONGO_INITDB_ROOT_USERNAME', 'admin')}:{os.getenv('MONGO_INITDB_ROOT_PASSWORD', 'admin123')}@mongo:27017/"
REDIS_HOST = "redis"
REDIS_PORT = 6379
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "redis123")

print("Variables cargadas y hosts definidos.")

# --- Conexiones ---
def connect_neo4j():
    try:
        driver = GraphDatabase.driver(NEO4J_HOST, auth=(NEO4J_USER, NEO4J_PASSWORD))
        with driver.session() as s:
            s.run("RETURN 1")
        print("✅ Conectado a Neo4j")
        return driver
    except Exception as e:
        print(f"❌ Error al conectar a Neo4j: {e}")
        return None

def connect_mongo():
    try:
        client = MongoClient(MONGO_URI)
        client.server_info()
        print("✅ Conectado a MongoDB")
        return client
    except Exception as e:
        print(f"❌ Error al conectar a MongoDB: {e}")
        return None

def connect_redis():
    try:
        r = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD, decode_responses=True)
        r.ping()
        print("✅ Conectado a Redis")
        return r
    except Exception as e:
        print(f"❌ Error al conectar a Redis: {e}")
        return None

neo4j_driver = connect_neo4j()
mongo_client = connect_mongo()
redis_client = connect_redis()

DB_NAME = "base_proyecto"
db = mongo_client[DB_NAME] if mongo_client else None

# Cargo constantes
DESTINO_ID_A_CIUDAD = {
    1: "Bariloche",
    2: "Cancún",
    3: "Madrid",
    4: "Roma",
    5: "Mendoza",
    6: "Ushuaia",
    7: "Puerto Iguazú",
    8: "Salta",
    9: "Río de Janeiro",
    10: "Punta Cana",
    11: "El Calafate",
    12: "Santiago",
    13: "Nueva York",
    14: "Tucumán",
    15: "Mar del Plata"
}

Esperando servicios (5s)...
Variables cargadas y hosts definidos.
✅ Conectado a Neo4j
✅ Conectado a MongoDB
✅ Conectado a Redis


# PARTE A: Carga inicial de Datasets

## 1. Carga de Datos Base y Limpieza
    Se cargan los datos de los archivos JSON requeridos y se limpian las bases de datos.

In [672]:
DATA_DIR = "data"
required_files = ["usuarios.json", "destinos.json", "hoteles.json", "actividades.json", "reservas.json"]
data = {}
missing = []

for f_name in required_files:
    f_path = os.path.join(DATA_DIR, f_name)
    if os.path.exists(f_path):
        with open(f_path, encoding="utf-8") as f:
            data[f_name.split('.')[0]] = json.load(f)
    else:
        missing.append(f_name)

if missing:
    print(f"⚠️ No se encontraron los siguientes archivos en '{DATA_DIR}/': {', '.join(missing)}")
    print("Por favor, asegúrate de tener todos los archivos JSON necesarios.")
else:
    print("✅ Archivos de datos cargados correctamente.")
    print("-" * 30)

# --- Limpieza de Bases de Datos ---
if neo4j_driver:
    with neo4j_driver.session() as s:
        s.run("MATCH (n) DETACH DELETE n")
        print("✅ Neo4j limpio.")

if mongo_client:
    mongo_client.drop_database(DB_NAME)
    print(f"✅ MongoDB ({DB_NAME}) limpio.")

if redis_client:
    for key in redis_client.scan_iter("tp_*"): 
        redis_client.delete(key)
    print("✅ Claves temporales de Redis limpias.")

✅ Archivos de datos cargados correctamente.
------------------------------
✅ Neo4j limpio.
✅ MongoDB (base_proyecto) limpio.
✅ Claves temporales de Redis limpias.


## 2. Carga en MongoDB, Redis y Neo4j
Se distribuyen los datos:

    - **MongoDB:** Usuarios, Destinos, Hoteles, Actividades y Reservas Concretadas.
    - **Redis:** Reservas Pendientes (temporal) y Usuarios Conectados (set).
    - **Neo4j:** Nodos (Usuarios, Destinos) y Relaciones Sociales y de Visita.

In [673]:
if data:
    # Carga en MongoDB (Datos permanentes)
    db.usuarios.insert_many(data["usuarios"])
    db.destinos.insert_many(data["destinos"])
    db.hoteles.insert_many(data["hoteles"])
    db.actividades.insert_many(data["actividades"])
    
    # Filtrar Reservas: Concretadas a Mongo
    reservas_concretadas = [
        r for r in data["reservas"] 
        if r["estado"] == "Confirmada"
    ]
    if reservas_concretadas:
        db.reservas.insert_many(reservas_concretadas)
    
    print("✅ Datos base y Reservas concretadas cargados en MongoDB.")

    # Carga en Redis (Datos temporales: Caché y Reservas Pendientes)
    if redis_client:
        # Reservas Temporales (Pendientes)
        reservas_pendientes = [
            r for r in data["reservas"] 
            if r["estado"] == "Pendiente"
        ]
        for r in reservas_pendientes:
            clave_pend = f"viajes_reserva:pendiente:{r['reserva_id']}"
            r_str = {k: str(v) for k, v in r.items()}
            redis_client.hset(clave_pend, mapping=r_str)
            redis_client.expire(clave_pend, random.randint(30, 120))
        print(f"✅ {len(reservas_pendientes)} Reservas Pendientes cargadas en Redis.")

        # Usuarios Conectados
        usuarios_conectados = random.sample([usu["nombre"] for usu in data["usuarios"]], k=random.randint(5, 25))
        for nombre in usuarios_conectados:
            redis_client.sadd("viajes_usuarios_conectados", nombre)
        print(f"✅ {len(usuarios_conectados)} Usuarios Conectados cargados en Redis.")

    # Carga en Neo4j (Nodos y Relaciones)
    # Nodos 
    if neo4j_driver:
        with neo4j_driver.session() as s:
            # Crear Constraints
            s.run("CREATE CONSTRAINT IF NOT EXISTS FOR (u:Usuario) REQUIRE u.usuario_id IS UNIQUE")
            s.run("CREATE CONSTRAINT IF NOT EXISTS FOR (d:Destino) REQUIRE d.destino_id IS UNIQUE")

            # Nodos
            for u in data["usuarios"]:
                s.run("MERGE (u:Usuario {usuario_id:$id, nombre:$nombre})", id=u["usuario_id"], nombre=u["nombre"])
            for d in data["destinos"]:
                s.run("MERGE (d:Destino {destino_id:$id, ciudad:$ciudad})", id=d["destino_id"], ciudad=d["ciudad"])

    # Relaciones VISITÓ basadas en Reservas Confirmadas Pasadas
    hoy = datetime.now().date()
    reservas_validas = [
        r for r in data.get("reservas", [])
        if r.get("estado") == "Confirmada" and datetime.strptime(r["fecha_reserva"], "%Y-%m-%d").date() <= hoy
        ]
  
    with neo4j_driver.session() as s:
        if reservas_validas not in (None, []):
            for r in reservas_validas:
                u_id = r["usuario_id"]
                d_id = r["destino_id"]
                s.run(
                        """
                        MATCH (u:Usuario {usuario_id:$u}), (d:Destino {destino_id:$d})
                        MERGE (u)-[:VISITO]->(d)
                        """,
                        u=u_id, d=d_id  
                    )
            print(f"✅ {len(reservas_validas)} relaciones VISITÓ creadas en Neo4j basadas en reservas confirmadas pasadas.")
            df_visito = pd.DataFrame(reservas_validas)
            df_visito['Nombre Usuario'] = df_visito['usuario_id'].map({u["usuario_id"]: u["nombre"] for u in data["usuarios"]})
            df_visito['Ciudad Destino'] = df_visito['destino_id'].map(DESTINO_ID_A_CIUDAD)
            display(df_visito[["reserva_id", "fecha_reserva", "estado", "Nombre Usuario", "Ciudad Destino"]].sort_values("Nombre Usuario"))
        else:
            print("⚠️ No hay reservas confirmadas pasadas para crear relaciones VISITÓ.")

        # Relaciones Sociales (AMIGO_DE, FAMILIAR_DE) - Evitando duplicados e inversas
        tipos_relacion = ["AMIGO_DE", "FAMILIAR_DE"] 
            
        # Intentamos crear el doble de relaciones que usuarios
        relaciones_creadas = []

        for _ in range(len(data["usuarios"]) * 2):
                u1 = random.choice(data["usuarios"])
                u2 = random.choice(data["usuarios"])
                u1_id = u1["usuario_id"]
                u2_id = u2["usuario_id"]
                tipo = random.choice(tipos_relacion)

                # 1. Aseguramos que los IDs sean diferentes.
                if u1_id == u2_id:
                    continue
               
                query = f"""
                MATCH (u1:Usuario {{usuario_id: $u1_id}})
                MATCH (u2:Usuario {{usuario_id: $u2_id}})

                WHERE NOT (u1)-[:AMIGO_DE|FAMILIAR_DE]-(u2)
                
                MERGE (u1)-[:{tipo}]->(u2)
                """
                # Ejecutar la query con los parámetros
                resultado_relaciones = s.run(query, u1_id=u1_id, u2_id=u2_id)

                if resultado_relaciones is not None:
                # Almacenar los datos de la relación *creada*
                    relaciones_creadas.append({
                        "Usuario 1": u1["nombre"],
                        "Usuario 2": u2["nombre"],
                        "Tipo de Relación": tipo
                    })
        query_fix_bidireccional = """
            MATCH (a:Usuario)-[:AMIGO_DE]->(b:Usuario)
            WHERE NOT (b)-[:AMIGO_DE]->(a)
            MERGE (b)-[:AMIGO_DE]->(a)
            RETURN COUNT(*) AS relaciones_creadas
            """

        try:
            result = s.run(query_fix_bidireccional).single()
            cantidad_creadas = result["relaciones_creadas"] if result else 0
            print(f"🔄 Se aseguraron {cantidad_creadas} relaciones AMIGO_DE bidireccionales.")
        except Exception as e:
            print("⚠️ Error al asegurar bidireccionalidad:", e)

        df_relaciones = pd.DataFrame(relaciones_creadas)

        print(f"✅ {df_relaciones.shape[0]} Relaciones sociales (AMIGO_DE/FAMILIAR_DE) creadas.")

        pd.set_option('display.max_rows', 100)
        pd.set_option('display.max_colwidth', 500)

        display(df_relaciones.sort_values(["Tipo de Relación", "Usuario 1"]).reset_index(drop=True))

✅ Datos base y Reservas concretadas cargados en MongoDB.
✅ 13 Reservas Pendientes cargadas en Redis.
✅ 7 Usuarios Conectados cargados en Redis.
✅ 29 relaciones VISITÓ creadas en Neo4j basadas en reservas confirmadas pasadas.


,reserva_id,fecha_reserva,estado,Nombre Usuario,Ciudad Destino
28,50,2025-04-20,Confirmada,Agustina Benítez,Río de Janeiro
15,25,2025-03-01,Confirmada,Agustina Benítez,Tucumán
2,5,2024-06-25,Confirmada,Ana Torres,Mendoza
12,18,2024-08-25,Confirmada,Andrés Pereyra,Puerto Iguazú
26,47,2025-01-25,Confirmada,Bárbara Nuñez,Ushuaia
24,41,2025-07-15,Confirmada,Carla Gómez,Mar del Plata
18,31,2025-09-10,Confirmada,Delfina Giménez,Mendoza
11,17,2024-07-20,Confirmada,Diego Romero,Ushuaia
8,11,2024-01-01,Confirmada,Emilia Ríos,Bariloche
19,32,2025-10-15,Confirmada,Ezequiel Blanco,Ushuaia


🔄 Se aseguraron 54 relaciones AMIGO_DE bidireccionales.
✅ 98 Relaciones sociales (AMIGO_DE/FAMILIAR_DE) creadas.


,Usuario 1,Usuario 2,Tipo de Relación
0,Agustina Benítez,Valentina Gil,AMIGO_DE
1,Ana Torres,Diego Romero,AMIGO_DE
2,Andrés Pereyra,Sebastián Maradona,AMIGO_DE
3,Andrés Pereyra,Milagros Herrera,AMIGO_DE
4,Camilo Arias,Delfina Giménez,AMIGO_DE
5,Carla Gómez,Milagros Herrera,AMIGO_DE
6,Carlos Colapinto,Ana Torres,AMIGO_DE
7,Carlos Colapinto,Milagros Herrera,AMIGO_DE
8,Carlos Colapinto,Santiago Castro,AMIGO_DE
9,Carlos Colapinto,Laura Ruiz,AMIGO_DE


# PARTE B: Consultas

## 1. Funciones de Consulta (a–l)
Definición de las funciones *helper* y las consultas específicas para cada base de datos.

In [674]:
# --- Helper Functions ---

def query_neo4j(query, **params):
    """Función helper para ejecutar consultas en Neo4j."""
    if not neo4j_driver: return []
    with neo4j_driver.session() as s:
        result = s.run(query, **params)
        return [r for r in result]

def query_mongo(collection_name, query):
    """Función helper para ejecutar consultas en MongoDB."""
    if not db: return []
    return list(db[collection_name].find(query))

def query_redis(pattern):
    """Función helper para ejecutar consultas en Redis."""
    if not redis_client: return []
    keys = redis_client.keys(pattern)
    results = []
    for key in keys:
        results.append(redis_client.hgetall(key))
    return results

# --- Neo4j Queries ---

def consulta_a_usuarios_bariloche():
    """Consulta 2.a: Mostrar los usuarios que visitaron 'Bariloche'."""
    q = "MATCH (u:Usuario)-[:VISITO]->(d:Destino {ciudad:'Bariloche'}) RETURN u.nombre AS nombre, u.usuario_id AS id"
    pd_a = pd.DataFrame(query_neo4j(q)).rename(columns={0: "Usuario", 1: "ID Usuario"})
    return pd_a

def consulta_b_amigos_destinos(nombre="Juan López"):
    """Consulta 2.b: Amigos de Juan López que visitaron un destino que él también visitó.
      Recomendando los más populares que no hayan visitado aún."""

    q = """
    MATCH (u:Usuario {nombre:$nombre})

    // Obtener los destinos que NO debemos sugerir
    OPTIONAL MATCH (u)-[:VISITO|AMIGO_DE|FAMILIAR_DE*0..2]->(p:Usuario)-[:VISITO]->(d_no:Destino)
    WITH u, collect(DISTINCT d_no) AS no_sugerir_list

    // Calcular popularidad de todos los destinos (cuántos usuarios los visitaron)
    MATCH (d:Destino)<-[:VISITO]-(otros:Usuario)
    WITH u, no_sugerir_list, d, count(otros) AS popularidad

    // Excluir destinos ya conocidos
    WHERE NOT d IN no_sugerir_list

    RETURN 
        d.ciudad AS `Destino Sugerido`,
        popularidad AS `Cantidad de Visitas`
    ORDER BY popularidad DESC, `Destino Sugerido` ASC
    LIMIT 5
    """

    try:
        resultados = query_neo4j(q, {"nombre": nombre for _nombre in [nombre]})
        df_c = pd.DataFrame(resultados)
    except Exception as e:
        print("⚠️ Error ejecutando consulta Neo4j:", e)
        return pd.DataFrame()

    if df_c.empty:
        print(f"⚠️ No se encontraron destinos sugeridos para {nombre}.")
    else:
        print(f"✅ Destinos sugeridos para {nombre}:")
        display(df_c)

    return df_c

def consulta_d_recomendar_por_amigos(nombre):
    """Consulta 2.d: Recomendar destinos basados en viajes de amigos (no visitados por el usuario)."""
    q = """MATCH (u:Usuario {nombre:$nombre})-[:AMIGO_DE]->(a:Usuario)-[:VISITO]->(d:Destino)
           WHERE NOT (u)-[:VISITO]->(d)
           RETURN DISTINCT d.ciudad AS destino, d.destino_id AS destino_id LIMIT 5"""
    return pd.DataFrame(query_neo4j(q, nombre=nombre))

def consulta_l_reservas_por_usuario():
    """Consulta 2.l: Mostrar la cantidad de reservas concretadas de cada usuario (Mongo)."""
    if not db: return pd.DataFrame()
    q = [{'$group': {'_id': '$usuario_id', 'cantidad_reservas': {'$sum': 1}}},
         {'$sort': {'cantidad_reservas': -1}}]
    
    resultados = list(db.reservas.aggregate(q))
    
    # Unir con nombres de usuario de la colección 'usuarios'
    usuarios_df = pd.DataFrame(list(db.usuarios.find({}, {"usuario_id": 1, "nombre": 1, "_id": 0})))
    reservas_df = pd.DataFrame(resultados).rename(columns={'_id': 'usuario_id'})
    
    df_final = pd.merge(reservas_df, usuarios_df, on='usuario_id', how='left')
    return df_final[['nombre', 'cantidad_reservas']].fillna("Desconocido")


# --- MongoDB Queries ---

def consulta_e_hoteles_destinos_recomendados(df_recomendaciones):
    """Consulta 2.e: Listar los hoteles en los destinos recomendados del punto anterior (d)."""
    destinos_ids = set()

    if df_recomendaciones.empty:
        print("No hay destinos recomendados para mostrar hoteles.")
    
    for d in df_recomendaciones['destino_id']:
        destinos_ids.add(d)
    
    q = {"destino_id": {"$in": destinos_ids}}
    hoteles = query_mongo("hoteles", q)

    return pd.DataFrame(hoteles)[['nombre', DESTINO_ID_A_CIUDAD[d['destino_id']], 'precio', 'servicios']]

def get_pending_reservations_redis():
    """Consulta 2.f: Ver las reservas en proceso (Redis)."""
    if not redis_client: return []
    claves = redis_client.keys("tp_reserva:pendiente:*\n")
    reservas = []
    for clave in claves:
        reservas.append(redis_client.hgetall(clave))
    return reservas

def consulta_h_destinos_baratos(precio_max=100000):
    """Consulta 2.h: Mostrar los destinos con precio inferior a $100.000 (Mongo)."""
    q = {"precio_base": {"$lt": precio_max}}
    return pd.DataFrame(query_mongo("destinos", q))[['ciudad', 'pais', 'precio_base']]

def consulta_i_hoteles_destino(ciudad="Jujuy"):
    """Consulta 2.i: Mostrar todos los Hoteles de un destino (Mongo)."""
    q = {"ciudad": ciudad}
    return pd.DataFrame(query_mongo("hoteles", q))[['nombre', 'ciudad', 'precio', 'servicios']]

def consulta_j_cantidad_hoteles(ciudad="Bariloche"):
    """Consulta 2.j: Mostrar la cantidad de hoteles de un destino (Mongo)."""
    return db.hoteles.count_documents({"ciudad": ciudad}) if db else 0

def consulta_k_actividades_por_tipo(ciudad="Ushuaia", tipo="aventura"):
    """Consulta 2.k: Mostrar las actividades de una ciudad y tipo (Mongo)."""
    q = {"ciudad": ciudad, "tipo": tipo}
    return pd.DataFrame(query_mongo("actividades", q))[['nombre', 'ciudad', 'tipo', 'precio']]

# --- Redis Queries ---

def get_connected_users_redis():
    """Consulta 2.g: Listar los usuarios conectados actualmente (Redis)."""
    q = query_redis("viajes_usuarios_conectados")
    return [u for u in q]   

## 2. Resultados de Consultas Integradas (a–l)


In [675]:
def safe_find(collection, query=None):
    """Devuelve lista de documentos sin romper si no hay Mongo."""
    try:
        if 'db' in globals() and hasattr(db, collection):
            col = getattr(db, collection)
            return list(col.find(query or {}))
    except Exception:
        pass
    return globals().get(collection, [])

def print_section(title):
    print("\n" + "="*80)
    print(f"🔹 {title}")
    print("="*80)


    a) Mostrar los usuarios que visitaron “Bariloche”.

In [676]:
df_a = consulta_a_usuarios_bariloche()
if df_a is not None:
    display(df_a.head())

,Usuario,ID Usuario
0,Emilia Ríos,11
1,Juan López,2


    b) Mostrar los amigos de Juan López que visitaron algún destino que visitó él, mostrando el nombre del usuario y el destino.

In [677]:
def obtener_amigos_y_destinos_visitados(nombre_referencia):
    """
    Obtiene los amigos de un usuario (busca AMIGO_DE en ambos sentidos)
    y los destinos que cada amigo visitó. Devuelve DataFrame.
    Esta versión es robusta ante distintos formatos de retorno de `query_neo4j`.
    """
    # Cypher: buscamos AMIGO_DE en ambos sentidos y los destinos (distinct)
    q = """
    MATCH (user:Usuario {nombre: $nombre})
    MATCH (user)-[:AMIGO_DE]-(amigo:Usuario)
    OPTIONAL MATCH (amigo)-[:VISITO]->(d:Destino)
    RETURN amigo.nombre AS Amigo, collect(DISTINCT d.ciudad) AS DestinosVisitados
    """

    # 1) Ejecutar la consulta intentando usar parámetros si la función lo soporta
    resultados = None
    last_exception = None
    try:
        # intento 1: si query_neo4j soporta params (dict)
        resultados = query_neo4j(q, {"nombre": nombre_referencia})
    except TypeError as e:
        # la función no acepta params en segundo arg -> intentar sin params (inyectar seguro)
        try:
            # escape muy básico de comillas simples en el nombre (para no romper la consulta)
            safe_nombre = nombre_referencia.replace("'", "\\'")
            q2 = q.replace("$nombre", f"'{safe_nombre}'")
            resultados = query_neo4j(q2)
        except Exception as e2:
            last_exception = e2
    except Exception as e:
        last_exception = e

    # si sigue None o hubo excepción, intentamos ejecutar una variante simple para depurar
    if resultados is None:
        print("⚠️ No se obtuvo resultado directo de la consulta. Detalle de excepción:", last_exception)
        print("Ejecutando consulta de debug (muestra algunas relaciones AMIGO_DE)...")
        try:
            debug_q = "MATCH (u:Usuario)-[:AMIGO_DE]-(a:Usuario) RETURN u.nombre AS Usuario, a.nombre AS Amigo LIMIT 25"
            debug_res = None
            try:
                debug_res = query_neo4j(debug_q)
            except TypeError:
                debug_res = query_neo4j(debug_q)  # reintento en cualquier forma
            print("Resultado debug (primeras filas):")
            # mostrar raw debug_res de forma legible
            if not debug_res:
                print(" -> No se encontraron relaciones AMIGO_DE en la base (resultado vacío).")
            else:
                for i, r in enumerate(debug_res[:10]):
                    print(f"  {i+1}. {r}")
        except Exception as e:
            print("  ⚠️ Error al ejecutar debug:", e)

        return pd.DataFrame()  # salimos, no hay nada que procesar

    # 2) Normalizar el formato de 'resultados' a lista de dicts con claves 'Amigo' y 'DestinosVisitados'
    rows = []
    try:
        if isinstance(resultados, dict):
            # a veces devuelve un diccionario single -> convertir a lista
            posibles = [resultados]
        else:
            posibles = list(resultados)

        for item in posibles:
            if item is None:
                continue
            # neo4j.Record suele comportarse como mapping, pero mejor convertir a dict si se puede
            try:
                if hasattr(item, "keys") and not isinstance(item, dict):
                    # neo4j.Record -> dict(item)
                    d = dict(item)
                elif isinstance(item, tuple) and len(item) == 2:
                    # posible forma (Amigo, DestinosVisitados)
                    d = {"Amigo": item[0], "DestinosVisitados": item[1]}
                elif isinstance(item, dict):
                    d = item
                else:
                    # último recurso: intentar indexación por posición
                    try:
                        d = {"Amigo": item[0], "DestinosVisitados": item[1]}
                    except Exception:
                        # no reconocido -> guardar crudo
                        d = {"Amigo": None, "DestinosVisitados": None}
            except Exception:
                d = {"Amigo": None, "DestinosVisitados": None}

            # normalizar claves (puede venir 'amigo.nombre' o 'Amigo', etc.)
            # buscamos la clave que contenga 'amig' y la que contenga 'Destin'
            amigo_val = None
            destinos_val = None
            for k, v in d.items():
                kn = str(k).lower()
                if "amig" in kn or "friend" in kn:
                    amigo_val = v
                if "dest" in kn or "ciudad" in kn:
                    destinos_val = v
            # fallback por claves exactas
            amigo_val = amigo_val if amigo_val is not None else d.get("Amigo") or d.get("amigo") or d.get("usuario")
            destinos_val = destinos_val if destinos_val is not None else d.get("DestinosVisitados") or d.get("destinos") or d.get("collect(d.ciudad)")

            rows.append({"Amigo": amigo_val, "DestinosVisitados": destinos_val})
    except Exception as e:
        print("⚠️ Error normalizando resultados:", e)
        return pd.DataFrame()

    # 3) Construir DataFrame
    df = pd.DataFrame(rows)

    # Si quedó vacío, mostramos qué vino en raw para depuración
    if df.empty:
        print("⚠️ Después de normalizar, no hay filas válidas. Raw resultados (primeros 10):")
        for i, r in enumerate(posibles[:10]):
            print(f"  {i+1}. {r}")
        return pd.DataFrame()

    # 4) Post-procesado: limpiar valores, desanidar listas, etc.
    def _format_destinos(x):
        # casos posibles: None, [], ['Bariloche'], 'Bariloche'
        if x is None:
            return "Ninguno"
        if isinstance(x, (list, tuple)):
            # eliminar elementos falsos
            x2 = [str(e) for e in x if e]
            return ", ".join(x2) if x2 else "Ninguno"
        # si es string
        s = str(x)
        if s.strip() == "":
            return "Ninguno"
        return s

    df["Amigo"] = df["Amigo"].astype(object).fillna("Desconocido")
    df["Destinos Visitados"] = df["DestinosVisitados"].apply(_format_destinos)

    # Renombrar columna Amigo para mostrar contextualmente
    col_amigo = f"Amiga/o de {nombre_referencia}"
    df = df.rename(columns={"Amigo": col_amigo})

    return df
# -------------------------
# Probamos
# -------------------------
nombre_usuario = "Juan López"
df_amigos_visitados = obtener_amigos_y_destinos_visitados(nombre_usuario)

if df_amigos_visitados.empty:
    print(f"⚠️ Resultado vacío para '{nombre_usuario}'. Revisa los mensajes de debug anteriores.")
else:
    print(f"✅ Amigos de {nombre_usuario} y destinos visitados:")
    display(df_amigos_visitados[["Amiga/o de Juan López", "Destinos Visitados"]])

✅ Amigos de Juan López y destinos visitados:


,Amiga/o de Juan López,Destinos Visitados
0,Federico Diaz,Salta
1,María Pérez,Cancún
2,Sebastián Maradona,Ninguno
3,Camilo Arias,Ninguno
4,Laura Ruiz,Puerto Iguazú


In [678]:
df_b = consulta_b_amigos_destinos()
if df_b is not None:
    display(df_b.head())

⚠️ Error ejecutando consulta Neo4j: query_neo4j() takes 1 positional argument but 2 were given


""


    c) Sugerir destinos a un usuario que no haya visitado él ni sus amigos.

In [679]:
df_c = consulta_c_sugerir_destinos("Juan López")
if df_c is not None:
    display(df_c)

,Destino Sugerido,ID Destino
0,Madrid,3
1,Nueva York,13
2,Punta Cana,10
3,Roma,4
4,Santiago,12
5,Tucumán,14
6,Ushuaia,6


    d) Recomendar destinos basados en viajes de amigos.

In [ ]:
df_d = consulta_d_recomendar_por_amigos("Juan López")
if df_d is not None:    
    display(df_d)

""


    e) Listar los hoteles en los destinos recomendados del punto anterior.

In [681]:
df_e = consulta_e_hoteles_destinos_recomendados(df_d)
display(df_e.head())

No hay destinos recomendados para mostrar hoteles.


KeyError: 'destino_id'

    f) Ver las reservas en proceso, es decir, aquellas que aún no están concretadas.

In [ ]:
df_f = consulta_f_reservas_en_proceso()
display(df_f.head())

    g) Listar los usuarios conectados actualmente.

In [ ]:
df_g = consulta_g_usuarios_conectados()
display(df_g)

    h) Mostrar los destinos con precio inferior a $100.000.

In [ ]:
df_h = consulta_h_destinos_baratos(100000)
display(df_h.head())

    i) Mostrar todos los hoteles de “Jujuy”.

In [ ]:
df_i = consulta_i_hoteles_destino("Jujuy")
display(df_i.head())

    j) Mostrar la cantidad de hoteles de un destino que elija el usuario.

In [ ]:
cantidad_hoteles = consulta_j_cantidad_hoteles("Bariloche")
print(f"Cantidad de hoteles en Bariloche: {cantidad_hoteles}")

    k) Mostrar las actividades de “Ushuaia” del tipo “aventura”.

In [ ]:
df_k = consulta_k_actividades_por_tipo("Ushuaia", "aventura")
display(df_k.head())

    l) Mostrar la cantidad de reservas concretadas de cada usuario (mostrar el nombre del usuario y la cantidad).

In [ ]:
df_l = consulta_l_reservas_por_usuario()
display(df_l.head())

# PARTE D: Estadística. Gráficos y visualizaciones (m)

    1. Destino más visitado. 
    2. Hotel más barato.   
    3. Actividad más popular.

In [ ]:
# ---------- 1) Destino más visitado ----------
# Intentamos calcularlo desde la colección 'reservas' en MongoDB.
# Si no existe, usamos Neo4j contando relaciones VISITO por destino.
def calcular_destinos_mas_visitados(top_n=10):
    # Primero, intentar por MongoDB (colección 'reservas')
    destinos_agg = []
    if "reservas" in db.list_collection_names():
        pipeline = [
            {"$group": {"_id": "$destino_id", "cantidad": {"$sum": 1}}},
            {"$sort": {"cantidad": -1}}
        ]
        destinos_agg = list(db.reservas.aggregate(pipeline))
        # Mapear destino_id a ciudad
        if destinos_agg:
            # recuperar destinos
            id_map = {d["destino_id"]: d["ciudad"] for d in db.destinos.find({}, {"_id":0,"destino_id":1,"ciudad":1})}
            for doc in destinos_agg:
                doc["ciudad"] = id_map.get(doc["_id"], f"destino_{doc['_id']}")
    # Si no hay datos en reservas, fallback a Neo4j (contar VISITO)
    if not destinos_agg:
        try:
            q = """
            MATCH (u:Usuario)-[:VISITO]->(d:Destino)
            RETURN d.destino_id AS destino_id, d.ciudad AS ciudad, count(u) AS cantidad
            ORDER BY cantidad DESC
            """
            with neo4j_driver.session() as s:
                res = s.run(q)
                destinos_agg = [{"_id": r["destino_id"], "cantidad": r["cantidad"], "ciudad": r["ciudad"]} for r in res]
        except Exception:
            destinos_agg = []
    # DataFrame para graficar y devolver
    if not destinos_agg:
        return pd.DataFrame(columns=["destino_id","ciudad","cantidad"])
    df = pd.DataFrame([{"destino_id": d["_id"], "ciudad": d.get("ciudad", f"destino_{d['_id']}"), "cantidad": d["cantidad"]} for d in destinos_agg])
    df = df.sort_values("cantidad", ascending=False).head(top_n).reset_index(drop=True)
    return df

df_destinos = calcular_destinos_mas_visitados(top_n=10)

# ---------- 1.A Gráfico: barra (conteo por destino) ----------
if not df_destinos.empty:
    plt.figure(figsize=(10,4), dpi=120)
    plt.bar(df_destinos["ciudad"], df_destinos["cantidad"])
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Cantidad de reservas / visitas")
    plt.title("Reservas por destino (top)")
    plt.grid(axis="y", linestyle="--", linewidth=0.5, alpha=0.6)
    plt.tight_layout()
    plt.show()



In [ ]:

# ---------- 2) Hotel más barato ----------
# Buscamos el hotel con menor precio en la colección 'hoteles' de MongoDB (si existe)
def obtener_hotel_mas_barato(top_n=10):
    if "hoteles" not in db.list_collection_names():
        return pd.DataFrame()
    cursor = db.hoteles.find({}, {"_id":0, "hotel_id":1, "nombre":1, "ciudad":1, "precio":1}).sort("precio", 1).limit(top_n)
    docs = list(cursor)
    if not docs:
        return pd.DataFrame()
    df = pd.DataFrame(docs)
    return df

df_hoteles_baratos = obtener_hotel_mas_barato(top_n=12)

# ---------- 2.A Gráfico: barras (hoteles, destacar el más barato) ----------
if df_hoteles_baratos.empty:
    print("⚠️ No hay datos de hoteles para graficar.")
else:
    # localizar el más barato
    idx_min = df_hoteles_baratos["precio"].idxmin()
    min_name = df_hoteles_baratos.loc[idx_min, "nombre"]
    min_price = df_hoteles_baratos.loc[idx_min, "precio"]

    plt.figure(figsize=(10,6), dpi=120)
    bars = plt.bar(df_hoteles_baratos["nombre"], df_hoteles_baratos["precio"])
    # Anotar el más barato
    for i, rect in enumerate(bars):
        h = rect.get_height()
        plt.annotate(f"{int(h):,}", xy=(rect.get_x() + rect.get_width()/2, h), xytext=(0,4), textcoords="offset points", ha="center", va="bottom", fontsize=8)
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Precio del hotel ($)")
    plt.title(f"Hoteles más económicos (top) — hotel más barato: {min_name} (${int(min_price):,})")
    plt.grid(axis="y", linestyle="--", linewidth=0.5, alpha=0.6)
    plt.tight_layout()
    plt.show()


In [ ]:

# ---------- 3) Actividad más popular ----------
# Estrategia:
# - Si existe colección 'reservas_actividades' (vínculo explícito), usarla.
# - Si no existe, usar como proxy: contar visitas a la ciudad de cada actividad (Neo4j VISITO) y sumar por actividad en esa ciudad.
def calcular_actividad_mas_popular():
    coll_names = db.list_collection_names()
    candidate = None
    for name in ["reservas_actividades", "res_actividades", "reservas_actividad"]:
        if name in coll_names:
            candidate = name
            break
    if candidate:
        pipeline = [
            {"$group": {"_id": "$actividad_id", "cantidad": {"$sum": 1}}},
            {"$sort": {"cantidad": -1}}
        ]
        agg = list(db[candidate].aggregate(pipeline))
        if agg:
            id_map = {a["actividad_id"]: a["nombre"] for a in db.actividades.find({}, {"_id":0,"actividad_id":1,"nombre":1})}
            df = pd.DataFrame([{
                "actividad_id": a["_id"],
                "cantidad": a["cantidad"],
                "nombre": id_map.get(a["_id"], f"act_{a['_id']}")
            } for a in agg])
            return df

    # Fallback: usar VISITO en Neo4j como proxy de popularidad
    try:
        q = """
        MATCH (u:Usuario)-[:VISITO]->(d:Destino)
        RETURN d.ciudad AS ciudad, count(u) AS visitas
        """
        with neo4j_driver.session() as s:
            res = s.run(q)
            visitas_por_ciudad = {r["ciudad"]: r["visitas"] for r in res}
    except Exception:
        visitas_por_ciudad = {}

    activities = list(db.actividades.find({}, {"_id":0, "actividad_id":1, "nombre":1, "ciudad":1}))
    if not activities:
        return pd.DataFrame()

    rows = []
    for act in activities:
        city_visits = visitas_por_ciudad.get(act["ciudad"], 0)
        rows.append({
            "actividad_id": act["actividad_id"],
            "nombre": act["nombre"],
            "ciudad": act["ciudad"],
            "score": city_visits
        })
    df = pd.DataFrame(rows)
    df = df.sort_values("score", ascending=False).reset_index(drop=True)
    return df


df_actividades_pop = calcular_actividad_mas_popular()

# ---------- 3.A Gráfico: torta (popularidad de actividades) ----------
if df_actividades_pop.empty:
    print("⚠️ No hay datos para calcular popularidad de actividades.")
else:
    # Determinar qué columna usar (cantidad o score)
    if "cantidad" in df_actividades_pop.columns:
        valores = df_actividades_pop["cantidad"].astype(int)
        etiquetas = df_actividades_pop["nombre"]
    else:
        valores = df_actividades_pop["score"].astype(int)
        etiquetas = df_actividades_pop["nombre"]

    # Tomar top 10 actividades más populares
    top_n = min(10, len(valores))
    valores = valores[:top_n]
    etiquetas = etiquetas[:top_n]

    # Si hay muchas, agrupar las menores en "Otras"
    if len(etiquetas) > 8:
        top_k = 7
        etiquetas_top = etiquetas[:top_k].tolist()
        valores_top = valores[:top_k].tolist()
        restantes = valores[top_k:]
        etiquetas_top.append("Otras")
        valores_top.append(restantes.sum())
        etiquetas, valores = etiquetas_top, valores_top

    plt.figure(figsize=(7,7), dpi=120)
    plt.pie(
        valores,
        labels=etiquetas,
        autopct=lambda pct: f"{pct:.1f}%\n({int(round(pct/100*sum(valores)))})",
        startangle=90
    )
    plt.title("Actividades más populares (top)", fontsize=12)
    plt.axis("equal")  # mantener proporción circular
    plt.tight_layout()
    plt.show()




In [ ]:
# ---------- Resumen textual de las 3 métricas ----------
print("\n" + "="*60)
if not df_destinos.empty:
    top = df_destinos.iloc[0]
    print(f"Destino más visitado: {top['ciudad']} — {int(top['cantidad'])} visitas/reservas (top).")
else:
    print("Destino más visitado: no disponible.")

if not df_hoteles_baratos.empty:
    cheapest = df_hoteles_baratos.iloc[0]
    print(f"Hotel más barato: {cheapest['nombre']} ({cheapest['ciudad']}) — ${int(cheapest['precio']):,}")
else:
    print("Hotel más barato: no disponible.")

if not df_actividades_pop.empty:
    top_act = df_actividades_pop.iloc[0]
    # si existe 'cantidad' la mostramos, si no mostramos 'score'
    if "cantidad" in df_actividades_pop.columns:
        cnt = int(top_act["cantidad"])
    else:
        cnt = int(top_act["score"])
    print(f"Actividad más popular: {top_act['nombre']} ({top_act.get('ciudad','-')}) — {cnt} puntos/veces.")
else:
    print("Actividad más popular: no disponible.")
print("="*60)

# PARTE D: Realizar modificación en los datos

    - Incrementar el precio de las actividades de Tucuman en 5% 
    - Agregar al hotel id=1 el servicio de SPA 
    - Eliminar el destino que desee 
    - Eliminar un usuario que desee 
    - Eliminar las relaciones AMIGO_DE para un usuario que quiera. 

In [ ]:
def modificar_precio_actividades(ciudad, pct):
    """Incrementar precio de actividades en una ciudad (MongoDB)"""
    res = db.actividades.update_many({"ciudad": ciudad}, {"$mul": {"precio": 1 + pct}})
    print(f"Incrementadas {res.modified_count} actividades en {ciudad} en un {pct*100}%. (Mongo)")

def agregar_servicio_hotel(hotel_id, servicio):
    """Agregar un servicio a un hotel (MongoDB)"""
    res = db.hoteles.update_one({"hotel_id": hotel_id}, {"$addToSet": {"servicios": servicio}})
    print(f"Servicio '{servicio}' agregado al hotel {hotel_id}. Modificados: {res.modified_count} (Mongo)")

def eliminar_usuario(nombre):
    """Eliminar un usuario de MongoDB y sus nodos/relaciones en Neo4j (Integrado)"""
    
    # 1. Eliminar de Neo4j
    q_neo4j = "MATCH (u:Usuario {nombre:$nombre}) DETACH DELETE u"
    with neo4j_driver.session() as s:
        s.run(q_neo4j, nombre=nombre)
    print(f"Usuario {nombre} eliminado de Neo4j (Nodos y Relaciones).")
    
    # 2. Eliminar de MongoDB
    db.usuarios.delete_one({"nombre": nombre})
    print(f"Usuario {nombre} eliminado de MongoDB (Colección Usuarios).")
    

# Ejecución de Modificaciones
print("\n--- Modificaciones ---\n")
modificar_precio_actividades("Bariloche", 0.1)
agregar_servicio_hotel(hotel_id=1, servicio="Spa")
eliminar_usuario("Juan López")

## PARTE F. Cierre de Conexiones
    Se cierran las conexiones a las bases de datos.

In [ ]:
if neo4j_driver:
    neo4j_driver.close()
if mongo_client:
    mongo_client.close()
if redis_client:
    redis_client.close()
print("🔒 Conexiones cerradas correctamente.")